# Installer les librairies nécessaires

In [ ]:

import subprocess
import sys

required_libs = [
    "pandas",
    "numpy",
    "streamlit",
    "ta",
    "matplotlib",
    "scikit-learn"
]

for lib in required_libs:
    try:
        __import__(lib)
        print(f"✅ {lib} est déjà installé.")
    except ImportError:
        print(f"📦 Installation de {lib}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib])


✅ pandas est déjà installé.
✅ numpy est déjà installé.
✅ streamlit est déjà installé.
✅ ta est déjà installé.
✅ matplotlib est déjà installé.
📦 Installation de scikit-learn...
Defaulting to user installation because normal site-packages is not writeable


# Lancer pour importer les données

In [6]:
import os
import time
import pandas as pd
from datetime import datetime
from binance.client import Client

# Connexion à l'API Binance 
client = Client()

# Répertoire pour les fichiers CSV
save_dir = "Historique"
os.makedirs(save_dir, exist_ok=True)

# Paires et timeframes à gérer
pairs = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "DOGEUSDT"]
timeframes = {
    "1h": Client.KLINE_INTERVAL_1HOUR,
    "4h": Client.KLINE_INTERVAL_4HOUR,
    "1d": Client.KLINE_INTERVAL_1DAY
}
start_date_str = "1 Jan, 2017"

# Fonction pour télécharger tout l'historique d'une paire / interval
def get_full_history(symbol, interval, start_str):
    print(f"⬇️ Téléchargement : {symbol} - {interval}")
    all_data = []
    start_time = int(datetime.strptime(start_str, "%d %b, %Y").timestamp() * 1000)

    while True:
        try:
            data = client.get_klines(
                symbol=symbol,
                interval=interval,
                startTime=start_time,
                limit=1000
            )
            if not data:
                break

            df = pd.DataFrame(data, columns=[
                "open_time", "open", "high", "low", "close", "volume",
                "close_time", "quote_asset_volume", "number_of_trades",
                "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
            ])
            df["open_time"] = pd.to_datetime(df["open_time"], unit='ms')
            df["close_time"] = pd.to_datetime(df["close_time"], unit='ms')

            all_data.append(df)

            start_time = int(data[-1][6]) + 1  # close_time + 1ms
            time.sleep(1)
        except Exception as e:
            print(f"❌ Erreur : {e}")
            break

    return pd.concat(all_data) if all_data else pd.DataFrame()

# Fonction de mise à jour d'un fichier CSV
def update_history(symbol, interval_key):
    interval = timeframes[interval_key]
    filename = f"{symbol}_{interval_key}.csv"
    filepath = os.path.join(save_dir, filename)

    if os.path.exists(filepath):
        print(f"🔁 Mise à jour : {filename}")
        df_existing = pd.read_csv(filepath)
        df_existing["close_time"] = pd.to_datetime(df_existing["close_time"])

        last_timestamp = int(df_existing["close_time"].max().timestamp() * 1000) + 1
        new_data = []

        while True:
            try:
                data = client.get_klines(
                    symbol=symbol,
                    interval=interval,
                    startTime=last_timestamp,
                    limit=1000
                )
                if not data:
                    break

                df = pd.DataFrame(data, columns=[
                    "open_time", "open", "high", "low", "close", "volume",
                    "close_time", "quote_asset_volume", "number_of_trades",
                    "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
                ])
                df["open_time"] = pd.to_datetime(df["open_time"], unit='ms')
                df["close_time"] = pd.to_datetime(df["close_time"], unit='ms')

                new_data.append(df)

                last_timestamp = int(data[-1][6]) + 1
                time.sleep(1)
            except Exception as e:
                print(f"❌ Erreur MAJ : {e}")
                break

        if new_data:
            df_new = pd.concat(new_data)
            df_combined = pd.concat([df_existing, df_new])
            df_combined.drop_duplicates(subset="close_time", inplace=True)
            df_combined.to_csv(filepath, index=False)
            print(f"✅ Fichier mis à jour : {filename} ({len(df_combined)} lignes)")
        else:
            print("✅ Aucune nouvelle donnée.")
    else:
        print(f"📁 Création : {filename}")
        df = get_full_history(symbol, interval, start_date_str)
        if not df.empty:
            df.to_csv(filepath, index=False)
            print(f"✅ Fichier créé : {filename} ({len(df)} lignes)")
        else:
            print("❌ Aucune donnée trouvée.")

# Lancer le script pour toutes les paires et timeframes
for pair in pairs:
    for tf in timeframes.keys():
        update_history(pair, tf)


🔁 Mise à jour : BTCUSDT_1h.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : BTCUSDT_4h.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : BTCUSDT_1d.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : ETHUSDT_1h.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : ETHUSDT_4h.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : ETHUSDT_1d.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : BNBUSDT_1h.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : BNBUSDT_4h.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : BNBUSDT_1d.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : SOLUSDT_1h.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : SOLUSDT_4h.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : SOLUSDT_1d.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : DOGEUSDT_1h.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : DOGEUSDT_4h.csv
✅ Aucune nouvelle donnée.
🔁 Mise à jour : DOGEUSDT_1d.csv
✅ Aucune nouvelle donnée.


# Vérifier que tout les fichiers sont à jour

In [7]:

import os

required_files = [
    "Backend/app.py",
    "Backend/backtester.py",
    "Backend/metrics.py",
    "Backend/models.py",
    "Backend/optimizer.py",
    "Backend/strategies.py"
]

missing = [f for f in required_files if not os.path.exists(f)]

if missing:
    print(f"⚠️ Fichiers manquants dans 'Backend' : {missing}")
else:
    print("✅ Tous les fichiers nécessaires sont présents dans 'Backend'.")


✅ Tous les fichiers nécessaires sont présents dans 'Backend'.


# Démarrer l'application

In [ ]:
!streamlit run backend/app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.40.1.186:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
